In [1]:
import psycopg2

Some usefull SQL prommt:
- `CREATE database <db_name>`
- `DROP database <db_name>`

In [2]:
# Env Variables
DB_NAME = 'krannet'

In [3]:
# connection establishment
# connects to the standard postgress user
conn = psycopg2.connect(
   database="postgres",
    user='postgres',
    password='password',
    host='localhost',
    port= '5432'
)
 
conn.autocommit = True
 
# Creating a cursor object
cursor = conn.cursor()


In [4]:
 # Creates DB
cursor.execute(f'CREATE DATABASE {DB_NAME}')
print(f"Database '{DB_NAME}' has been created successfully !!");


DuplicateDatabase: database "krannet" already exists


In [5]:
# Closing the connection
conn.close()

____

In [6]:
# connection establishment
conn = psycopg2.connect(
   database=DB_NAME,
    user='postgres',
    password='password',
    host='localhost',
    port= '5432'
)

# Creating a cursor object
cursor = conn.cursor()

In [7]:
# create structure outer table with the meeting title and year

create_outer_table = f""" 
CREATE TABLE IF NOT EXISTS outer_table (
id SERIAL PRIMARY KEY,
title VARCHAR NOT NULL,
year CHAR(4)
)
"""



In [8]:
# create inner table with speaker and n. of interventions

create_inner_table = f""" 
CREATE TABLE IF NOT EXISTS outer_table (
id SERIAL PRIMARY KEY,
speaker VARCHAR(50) NOT NULL,
interventions SMALLINT NOT NULL
)
"""

In [9]:
# create outer table
cursor.execute(create_outer_table)
conn.commit()

In [ ]:
cursor.execute('SELECT * FROM outer_table')

In [ ]:
#conn.rollback()

# Variables to insert
title = 'meeting1'
year = 1990

# Parameterized query
cursor.execute('INSERT INTO outer_table (title, year) VALUES (%s, %s)', (title, year))
conn.commit()


In [10]:
cursor.execute('SELECT * FROM outer_table')
rows = cursor.fetchall()

rows





[(4, 'meeting1', '1990')]

In [ ]:
conn.rollback()
# Drop DB
# TODO how to drop the table?!
#cursor.execute(f'DROP TABLE outer_table')
#print(f"Table 'outher_table' has been dropped successfully !!");


conn.close()

___

In [ ]:
# Drop DB
cursor.execute(f'DROP DATABASE {DB_NAME}')
print(f"Database '{DB_NAME}' has been dropped successfully !!");

In [ ]:
# Closing the connection
conn.close()